# Autoencoder
This model using one of autoencoder abilities that allow to compare input and output. Based on that, autoencoder could determine if sample is normal or anomalous.

## Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split

## Data preparation

In [ ]:
dataframe = pd.read_csv('../dataset/signal_25k.csv', header=None)
raw_data = dataframe.values

labels = raw_data[:, -1]
data = raw_data[:, 0:-1]

train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=50)

min_val =  tf.reduce_min(raw_data)
max_val = tf.reduce_max(raw_data)

train_data = (train_data - min_val)/ (max_val - min_val)

train_data = tf.cast(train_data, tf.float32)
test_data = tf.cast(test_data, tf.float32)

train_labels = train_labels.astype(bool)
test_labels = test_labels.astype(bool)

normal_train_data = train_data[train_labels]
normal_test_data = test_data[test_labels]

anomalous_train_data = train_data[~train_labels]
anomalous_test_data = test_data[~test_labels]

## Set models parameters

In [ ]:
latent_dim = 64
input_dim = 100

## Define encoder model

In [ ]:
def build_encoder():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(latent_dim, activation='relu'))
    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dense(16, activation='relu'))
    model.add(tf.keras.layers.Dense(input_dim, activation='relu'))
    return model

## Define decoder model

In [ ]:
def build_decoder():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(16, activation='relu'))
    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(input_dim, activation='sigmoid'))
    return model

## Define autoencoder

In [ ]:
def build_autoencoder():
    encoder = build_encoder()
    decoder = build_decoder()

    encoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.002, beta_1=0.5), loss=tf.keras.losses.mse)
    decoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.002, beta_1=0.5), loss=tf.keras.losses.mse)

    encoder_input = tf.keras.layers.Input(shape=(input_dim,))
    autoencoder_input = encoder(encoder_input)
    autoencoder_output = decoder(autoencoder_input)
    autoencoder = tf.keras.Model(inputs=autoencoder_input, outputs=autoencoder_output)
    autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.002, beta_1=0.5), loss=tf.keras.losses.mse)

    return autoencoder

In [ ]:
autoencoder = build_autoencoder()

## Model fitting

In [ ]:
autoencoder.fit(normal_train_data, normal_train_data, epochs=100, batch_size=128, validation_data=(test_data, test_data), validation_batch_size=128)

## Save model

In [ ]:
dataframe = pd.read_csv('../dataset/signal_10k_val.csv', header=None)
random_dataframe = dataframe
raw_data = random_dataframe.values

labels = raw_data[:, -1]
data = raw_data[:, 0:-1]

data = (data - np.min(data))/(np.max(data) - np.min(data))
result = autoencoder.predict(data)


def mean_squared_error(y, y_pred):
    # Calculate the squared errors
    squared_errors = (y - y_pred) ** 2

    # Mean of the sum of squared errors
    mse = np.mean(squared_errors)

    return mse


mse = []

for i in range(len(result)):
    mse.append(mean_squared_error(data[i], result[i]))
correctly_recognized_samples_normal = 0
correctly_recognized_samples_anomalous = 0
temp = []
corr_max = [0, float('inf')]
for i in np.arange(0.001, 0.2, 0.00025):
    for j in range(0, len(mse)):
        if mse[j] > i and labels[j] == 0:
            correctly_recognized_samples_anomalous += 1
        if mse[j] < i and labels[j] == 1:
            correctly_recognized_samples_normal += 1
    if abs(correctly_recognized_samples_normal / 5000 - correctly_recognized_samples_anomalous / 5000) < corr_max[1]:
        corr_max = [i,
                    (correctly_recognized_samples_normal / 5000 + correctly_recognized_samples_anomalous / 5000) / 2]
        temp = [correctly_recognized_samples_normal, correctly_recognized_samples_anomalous]
    correctly_recognized_samples_normal = 0
    correctly_recognized_samples_anomalous = 0
print(corr_max)

In [ ]:
x = autoencoder.predict(data[100:101])

In [ ]:
import matplotlib.pyplot as plt
plt.plot(x)

In [ ]:
plt.plot(data[100:101])

In [ ]:
import matplotlib.pyplot as plt

# Define the x-axis values
x = [10000, 25000, 50000, 100000]

# Define the y-values for the two functions
y1 = [0.89, 0.89, 0.88, 0.87]
y2 = [0.56, 0.54, 0.56, 0.51]

# Create the plot
plt.figure(figsize=(10,6))

# Plot the two functions
plt.plot(x, y1, 'o-', label='Walidacja na zbiorze statycznym')
plt.plot(x, y2, 's-', label='Walidacja na strumieniu testowym')

# Annotate the y-values on the plot for each point
for xi, y1i, y2i in zip(x, y1, y2):
    plt.annotate(f"{y1i}", (xi, y1i), textcoords="offset points", xytext=(0,5), ha='center')
    plt.annotate(f"{y2i}", (xi, y2i), textcoords="offset points", xytext=(0,-15), ha='center')

    # Annotate x-values for each point
    offset = -30 if y1i < y2i else 15  # Adjust offset based on the lower y value
    plt.annotate(f"{xi}", (xi, 0), textcoords="offset points", xytext=(0,offset), ha='center', fontsize=9, color='gray')

# Set y-limits and x-limits
plt.ylim(0, 1.2)
plt.xlim(5000, 110000)

# Label the axes
plt.xlabel("Liczba próbek terningowych")
plt.ylabel("Dokładność modelu")

# Display the legend in the bottom left with an increased margin from the bottom
plt.legend(loc='lower left', bbox_to_anchor=(0, 0.1))

# Display the plot
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
autoencoder.save('../models/detectors/autoencoder_signal')

## Model testing